In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from news_summarizer.domain.documents import Article, Link
from news_summarizer.domain.base.nosql import _database

collection_link = _database.get_collection("link")
indexes = collection_link.find(**{})
indexes = list(indexes)

INFO:news_summarizer.database.mongo:Successfully connected to Mongo at: localhost


In [4]:
collection_link = _database.get_collection("article")
indexes = collection_link.find(**{})
indexes = list(indexes)

In [6]:
indexes

[{'_id': '0abcec78-cf22-4732-a49d-300bb2909ca9',
  'title': 'As ferramentas medievais utilizadas na reconstrução da catedral de Notre-Dame',
  'subtitle': 'Pouco mais de cinco anos após sofrer um incêndio que a devastou, a catedral de Notre-Dame de Paris é reaberta neste sábado.',
  'author': None,
  'publication_date': datetime.datetime(2024, 12, 7, 11, 23, 57, 911000),
  'content': ' Foram usados machados medievais criados pela Maison Luquet, ferreiro situado no leste da França, para o processo de restauração — Foto: Fabrice WITTNER – White Fall Lab \n Pouco mais de cinco anos após sofrer um incêndio que a devastou, a catedral de Notre-Dame de Paris será reaberta neste sábado (7/12) em uma cerimônia com a presença de chefes de Estados. \n Uma das proezas dessas "obras do século", como o restauro foi chamado pelo presidente francês, Emmanuel Macron, foi a reconstrução idêntica da estrutura de vigas de carvalho do telhado da catedral, que existia há 850 anos e foi carbonizada durante a

In [29]:
len(articles_urls)

440

In [30]:
links = Link.bulk_find(**{})
links_urls = [str(link.url) for link in links]

In [33]:
articles = Article.bulk_find(**{})
articles_urls = [str(link.url) for link in articles]

In [31]:
len(links_urls)

662

{'https://g1.globo.com/globonews/globonewsmais/video/queda-de-arvore-mata-duas-pessoas-em-santuario-de-bali-13173688.ghtml',
 'https://g1.globo.com/globonews/jornal-globonews-edicao-das-18/video/andreia-sadi-pf-faz-buscas-na-casa-do-ex-ministro-braga-netto-13184247.ghtml',
 'https://g1.globo.com/globonews/jornal-globonews/video/braga-netto-e-preso-no-inquerito-do-golpe-13184235.ghtml',
 'https://g1.globo.com/globonews/jornal-globonews/video/brasileira-juliana-rodrighiero-participou-de-restauracao-da-catedral-de-notre-dame-13166652.ghtml',
 'https://g1.globo.com/globonews/jornal-globonews/video/video-mostra-braga-netto-escoltado-por-policiais-ate-a-sede-da-pf-no-rj-13185371.ghtml',
 'https://g1.globo.com/pb/paraiba/noticia/2024/12/15/pai-ganha-olho-de-vidro-apos-pedido-emocionante-do-filho-em-cartinha-para-papai-noel.ghtml',
 'https://gcmais.com.br/noticias/2024/12/10/morte-de-crianca-no-ce-foi-causada-por-ameba-comedora-de-cerebro-diz-ministerio-da-saude/',
 'https://ge.globo.com/futeb

In [11]:
articles_ids

['0abcec78-cf22-4732-a49d-300bb2909ca9',
 '050762d9-2cc7-4647-9ec5-b7de1afcd038',
 '1b11e957-b758-408c-bd25-29ca66e21a77',
 '1cf84c70-6200-4bb1-977d-4f5205f5ecc9',
 '18b54be7-5a94-4c2e-9a74-1dfdbe7ba660',
 '173e14dd-7b73-4193-b63e-712489ce2c8b',
 '1351c1d4-3b19-4d19-ab72-a1e0af345bd7',
 '010a0f02-1fba-4522-8847-86a86eb7334c',
 '29e34115-452c-4ad6-ad1a-cb4d1da10473',
 '0ebdec04-676f-436a-a5e9-d2b8131ae156',
 '00b140fd-57a0-45de-a8c2-7cd32be48637',
 '117a3b8d-0dd0-4062-a968-93a8e9b77e20',
 '2fc9df05-81d1-48e6-9c86-8ac93ee35519',
 '3fbe2726-ff89-43ca-8624-b8ac9c764cf9',
 '9e9ad1bb-18b2-4866-870c-bf545db52be4',
 '1abbaf59-64ef-4d92-9000-992113840dab',
 '5352adf6-c4b8-4e2d-be13-2d77be863988',
 '13228534-ca4d-48b2-a2f4-9e6bee96b2a7',
 '3786aa58-7f75-41b4-8795-d3b2e6534cc4',
 '16db39be-a8c9-4836-9430-751d0e01f9f0',
 '0c6709ca-88f3-49f4-8652-49586162865a',
 '0fa9f589-abe4-4cde-a75e-270abce6815b',
 '0ec427cf-381b-4d79-978c-2bf09b623f06',
 '557cdddd-ff8c-4b0d-baf4-116d42c6a0c7',
 '479f6704-ceac-

In [10]:
sdfsdf

NameError: name 'sdfsdf' is not defined

In [59]:
from news_summarizer.database.mongo import MongoDatabaseConnector
from news_summarizer.config import settings

client = MongoDatabaseConnector()
database = client.get_database(settings.mongo.name)

collection = database[Article.get_collection_name()]

duplicates = _search_duplicates(collection, group_by="url")


INFO:news_summarizer.database.mongo:Successfully connected to Mongo at: localhost


In [60]:
instance = list(duplicates)[0]

In [73]:
docs = list(collection.find({"_id": {"$in": instance['ids']}}))

In [76]:
len(docs)

18

In [77]:
[doc["_id"] for doc in docs[1:]]

17

In [56]:
list(duplicates)

[{'_id': 'https://g1.globo.com/rj/rio-de-janeiro/noticia/2024/12/10/liberdade-socios-laboratorio-pacientes-hiv.ghtml',
  'count': 18,
  'ids': ['770aace1-da92-405b-a332-b212e047f7e9',
   'a40c8299-04d8-4836-8b90-d2002b8ef8a4',
   '0fcab3af-f5c4-4745-9e3b-c8a0ec8ac8b1',
   'd023c2a7-0a02-46d1-b31b-9fa79a2ff1e0',
   '2833fc7b-d144-4614-b7f6-def8812c8c4f',
   'd36eb244-71f5-4326-b55d-4750cfe6001e',
   '343f4f4f-cdbf-4bd8-ad73-193f7661925b',
   '65e188b8-cba0-4601-b7c8-7ca005ce8653',
   'db97ea35-1834-4bb7-ac03-4d0d6174cba5',
   'b194239c-0eea-4f84-a5a1-6a693a2f1f8b',
   '55769e6b-cd3f-4079-be2a-876fb54fa12a',
   '7835028f-3f96-481b-af78-eceadf68534a',
   'e4500179-4123-43f1-90b9-be7926828d09',
   '404fac64-3751-472f-b057-e98b9c30a1fc',
   '8ec902fd-fa6b-4741-a25a-071b4b46fa3f',
   'f6975bf1-e1d4-4729-b73a-95c75a1cfab6',
   'c030b926-b795-4369-a1b6-1af083a99bd0',
   '0952411f-5bc5-4202-b5f7-83dd56ea3cde']},
 {'_id': 'https://g1.globo.com/tecnologia/noticia/2024/12/10/tiktok-pede-suspensao-

In [48]:
sdf

NameError: name 'sdf' is not defined

In [ ]:
len(set(links_ids) - set(articles_ids))

NameError: name 'links_ids' is not defined

In [74]:
from datetime import datetime
from typing import Optional
from news_summarizer.domain.base.nosql import NoSQLBaseLink
from news_summarizer.scraper.base import BaseSeleniumScraper

from pydantic import AnyUrl, Field, field_serializer

class Link(NoSQLBaseLink):
    title: str = Field(..., description="The title of the link")
    url: AnyUrl = Field(description="The URL of the link")
    source: Optional[str] = Field(None, description="The source of the link")
    published_at: Optional[datetime] = Field(None, description="The publication date of the link")
    extracted_at: datetime = Field(
        default_factory=datetime.now,
        description="The timestamp when the link was extracted",
    )

    @field_serializer("url")
    def url_string(self, url: AnyUrl):
        return str(url)

    class Settings:
        name = "link"

from datetime import datetime

class Article(NoSQLBaseLink):
    title: str = Field(..., description="The title of the link")
    subtitle: str = Field(..., description="The subtitle of the link")
    author: str = Field(...,  description="The author")
    publication_date: Optional[datetime] = Field(None, description="The publication date of the link")
    content: str = Field(..., description="Content")
    url: AnyUrl = Field(description="The URL of the link")

    @field_serializer("url")
    def url_string(self, url: AnyUrl):
        return str(url)

    class Settings:
        name = "article"

INFO:WDM:====== WebDriver manager ======
INFO:WDM:Get LATEST chromedriver version for google-chrome
INFO:WDM:Get LATEST chromedriver version for google-chrome
INFO:WDM:Driver [/home/llya/.wdm/drivers/chromedriver/linux64/131.0.6778.108/chromedriver-linux64/chromedriver] found in cache
INFO:WDM:====== WebDriver manager ======
INFO:WDM:Get LATEST edgedriver version for Edge 131.0.2903
INFO:WDM:Get LATEST edgedriver version for Edge 131.0.2903
INFO:WDM:Driver [/home/llya/.wdm/drivers/edgedriver/linux64/131.0.2903.87/msedgedriver] found in cache


In [75]:
from bs4 import BeautifulSoup

class G1Scraper(BaseSeleniumScraper):
    model = Article

    def __init__(self) -> None:
        super().__init__()

    def extract(self, article_link: str, **kwargs) -> None:
        self.driver.get(article_link)
        self.driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        self.soup = BeautifulSoup(self.driver.page_source, "html.parser")
        if len(self.soup) == 0:
            raise ValueError("No elements scraped from link %s", article_link)

        title = self._extract_title(self.soup)
        author = self._extract_author(self.soup)
        subtitle = self._extract_subtitle(self.soup)
        content = self._extract_content(self.soup)
        publication_date = self._extract_publication_date(self.soup)

        intance = self.model(
            title=title,
            subtitle=subtitle,
            author=author,
            content=content,
            publication_date=publication_date,
            url=article_link,
        )

        intance.save()
    def _extract_title(self, soup: BeautifulSoup):
        title = soup.find("h1", class_="content-head__title").text
        return title

    def _extract_author(self, soup: BeautifulSoup):
        author = soup.find("a", class_="multi_signatures").text
        return author

    def _extract_subtitle(self, soup: BeautifulSoup):
        subtitle = soup.find("h2", class_="content-head__subtitle").text
        return subtitle

    def _extract_content(self, soup: BeautifulSoup):
        paragraphs = soup.select("div.mc-article-body p")
        content = "\n".join([p.text for p in paragraphs if len(p) > 0])
        return content

    def _extract_publication_date(self, soup: BeautifulSoup):
        publication_date = soup.find("time", itemprop="datePublished")["datetime"]
        return publication_date


class R7Scraper(BaseSeleniumScraper):
    model = Article

    def __init__(self) -> None:
        super().__init__()

    def extract(self, article_link: str, **kwargs) -> None:
        self.driver.get(article_link)
        self.driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        self.soup = BeautifulSoup(self.driver.page_source, "html.parser")
        if len(self.soup) == 0:
            raise ValueError("No elements scraped from link %s", article_link)
        
        title = self._extract_title(self.soup)
        author = self._extract_author(self.soup)
        subtitle = self._extract_subtitle(self.soup)
        content = self._extract_content(self.soup)
        publication_date = self._extract_publication_date(self.soup)

        intance = self.model(
            title=title,
            subtitle=subtitle,
            author=author,
            content=content,
            publication_date=publication_date,
            url=article_link,
        )

        intance.save()

    def _extract_title(self, soup: BeautifulSoup):
        title = soup.find(
            "h1",
            class_="base-font-primary dark:base-text-neutral-high-400 base-mb-xxxs base-text-xl base-font-semibold base-leading-xxl lg:base-leading-giant lg:base-text-xxl base-text-neutral-low-500",
        ).text
        return title

    def _extract_author(self, soup: BeautifulSoup):
        first_span = soup.find(
            "span",
            class_="article-text-editorial-color article-ml-quark article-mr-quark dark:!article-text-neutral-high-400",
        )
        author = first_span.find_next("span").text
        return author

    def _extract_subtitle(self, soup: BeautifulSoup):
        subtitle = soup.find(
            "h2",
            class_="base-font-primary dark:base-text-neutral-high-400 base-text-xxs base-font-bold base-leading-md sm:base-text-md sm:base-font-medium sm:base-leading-lg base-text-neutral-low-500",
        ).text
        return subtitle

    def _extract_content(self, soup: BeautifulSoup):
        article = soup.find("article", class_="b-article-body")
        paragraphs = article.find_all_next("span", class_="b-article-body__text")
        content = "\n".join([p.text for p in paragraphs if len(p) > 0])
        return content

    def _extract_publication_date(self, soup: BeautifulSoup):
        publication_date = soup.find("time", itemprop="datePublished")["datetime"]
        modified_date = soup.find("time", itemprop="dateModified")["datetime"]
        if modified_date is None:
            return publication_date
        return modified_date
    

class BandScraper(BaseSeleniumScraper):
    model = Article

    def __init__(self) -> None:
        super().__init__()

    def extract(self, article_link: str, **kwargs) -> None:
        self.driver.get(article_link)
        self.driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        self.soup = BeautifulSoup(self.driver.page_source, "html.parser")
        if len(self.soup) == 0:
            raise ValueError("No elements scraped from link %s", article_link)
        
        title = self._extract_title(self.soup)
        author = self._extract_author(self.soup)
        subtitle = self._extract_subtitle(self.soup)
        content = self._extract_content(self.soup)
        publication_date = self._extract_publication_date(self.soup)

        intance = self.model(
            title=title,
            subtitle=subtitle,
            author=author,
            content=content,
            publication_date=publication_date,
            url=article_link,
        )

        intance.save()

    def _extract_title(self, soup: BeautifulSoup):
        h1_element = soup.find(
            "h1",
            class_="cs-entry__title",
        )
        title = h1_element.find('span').text
        return title

    def _extract_author(self, soup: BeautifulSoup):
        author = soup.find(
            "span",
            class_="cs-meta-author-name",
        ).text
        return author

    def _extract_subtitle(self, soup: BeautifulSoup):
        subtitle = soup.find(
            "div",
            class_="cs-entry__subtitle",
        ).text
        return subtitle

    def _extract_content(self, soup: BeautifulSoup):
        article = soup.find("div", class_="cs-entry__content-wrap")
        paragraphs = article.find_all("p")
        content = "\n".join([p.text for p in paragraphs if len(p) > 0])
        return content

    def _translate_months(self, date_str: str):
        months_dict = {
            "janeiro": "January", "fevereiro": "February", "março": "March",
            "abril": "April", "maio": "May", "junho": "June",
            "julho": "July", "agosto": "August", "setembro": "September",
            "outubro": "October", "novembro": "November", "dezembro": "December"
        }

        # Traduz o nome do mês para inglês
        for month_pt, month_en in months_dict.items():
            date_str = date_str.replace(month_pt, month_en)

        return date_str


    def _extract_publication_date(self, soup: BeautifulSoup):
        date_string = soup.find("div", class_='cs-meta-date').text
        date_string = self._translate_months(date_string)
        date_format = "%B %d, %Y"
        publication_date = datetime.strptime(date_string, date_format)
        return publication_date

In [76]:
from news_summarizer.scraper import scraper_registry
from news_summarizer.scraper import ScraperExecutor

In [77]:
links = Link.bulk_find(**{})
links = [str(link.url) for link in links]

In [78]:
executor = ScraperExecutor(scraper_registry)

In [ ]:
executor.run(links[:20])

ERROR:news_summarizer.scraper.article_page:Error trying to extract title, 'NoneType' object has no attribute 'text'
ERROR:news_summarizer.scraper.article_page:Error while scraping link https://g1.globo.com/globonews/jornal-globonews/video/brasileira-juliana-rodrighiero-participou-de-restauracao-da-catedral-de-notre-dame-13166652.ghtml: Error trying to extract title
ERROR:news_summarizer.scraper.article_page:Error trying to extract author, 'NoneType' object has no attribute 'text'
ERROR:news_summarizer.scraper.article_page:Error trying to extract author, 'NoneType' object has no attribute 'text'


In [ ]:
collection_article = _database.get_collection("article")
indexes = collection_article.find(**{})
indexes = list(indexes)

In [ ]:
indexes

[{'_id': '875fca33-bc9b-4c95-9a47-2d39a9da5565',
  'title': 'Rebeldes, forças do governo, curdos e facções turcas: os grupos que disputam territórios na Síria',
  'subtitle': 'O país vive uma série de crises desde o início da guerra de 2011, quando opositores ao regime de Bashar al-Assad iniciaram protestos por democracia. Entenda como o país está dividido.',
  'author': 'Redação g1',
  'publication_date': datetime.datetime(2024, 12, 7, 15, 30, 57, 9000),
  'content': ' Rebeldes sírios avançam contra o regime de Bashar al-Assad — Foto: Omar Haj Kadour/ AFP \n Grupos rebeldes dominados por islamistas radicais do noroeste da Síria assumiram o controle de dezenas de localidades e duas das principais cidades do país em 10 dias, em uma ofensiva relâmpago que enfraqueceu ainda mais o regime de Bashar al-Assad. \n O avanço, facilitado pela retirada do Exército sírio em várias regiões, marca um ponto de virada na guerra da Síria, que começou em 2011 com a repressão de protestos pró-democracia.

In [ ]:

# SuperFastPython.com
# example of using a semaphore
from time import sleep
from random import random
from threading import Thread
from threading import Semaphore
 
# target function
def task(semaphore, number):
    # attempt to acquire the semaphore
    with semaphore:
        # process
        value = random()
        sleep(value)
        # report result
        print(f'Thread {number} got {value}')
 
# create a semaphore
semaphore = Semaphore(4)
# create a suite of threads
for i in range(10):
    worker = Thread(target=task, args=(semaphore, i))
    worker.start()
# wait for all workers to complete..

Thread 1 got 0.26979132276078077
Thread 4 got 0.2680321303617894
Thread 0 got 0.7349327955542244
Thread 3 got 0.773502219618814
Thread 2 got 0.8610218647067934
Thread 5 got 0.6249579095979747
Thread 8 got 0.36658455655124955
Thread 9 got 0.1666516001492402
Thread 6 got 0.787973507098034
Thread 7 got 0.7805049959260377


In [ ]:
# SuperFastPython.com
# example of bounding the number of tasks submitted to the thread pool
from time import sleep
from random import random
from threading import Semaphore
from concurrent.futures import ThreadPoolExecutor
 
# mock task that sleeps for a moment
def work(identifier):
    sleep(random())
    print(f'Done: {identifier}')
    return True
 
# callback for completed tasks
def task_complete_callback(future):
    global semaphore
    # release the semaphore
    semaphore.release()
 
# proxy for submitting tasks that imposes a limit on the queue size
def submit_proxy(function, *args, **kwargs):
    print('registrando nova tarefa')
    global semaphore, executor
    # acquire the semaphore, blocks if occupied
    semaphore.acquire()
    # submit the task normally
    future = executor.submit(function, *args, **kwargs)
    # add the custom done callback
    future.add_done_callback(task_complete_callback)
    return future
 
# number of works in the pool
n_workers = 2
# max number of queued tasks
n_queue = 10
# semaphore to limit the queue size to the pool
semaphore = Semaphore(n_queue)
# create the thread pool
with ThreadPoolExecutor(n_workers) as executor:
    # submit many tasks
    futures = [submit_proxy(work, i) for i in range(50)]
    # wait for all tasks to complete
    print('All tasks are submitted, waiting...')

registrando nova tarefa
registrando nova tarefa
registrando nova tarefa
registrando nova tarefa
registrando nova tarefa
registrando nova tarefa
registrando nova tarefa
registrando nova tarefa
registrando nova tarefa
registrando nova tarefa
registrando nova tarefa
Done: 1
registrando nova tarefa
Done: 2
registrando nova tarefa
Done: 0
registrando nova tarefa
Done: 3
registrando nova tarefa
Done: 5
registrando nova tarefa
Done: 4
registrando nova tarefa
Done: 6
registrando nova tarefa
Done: 7
registrando nova tarefa
Done: 8
registrando nova tarefa
Done: 9
registrando nova tarefa
Done: 11
registrando nova tarefa
Done: 10
registrando nova tarefa
Done: 12
registrando nova tarefa
Done: 14
registrando nova tarefa
Done: 13
registrando nova tarefa
Done: 15
registrando nova tarefa
Done: 16
registrando nova tarefa
Done: 18
registrando nova tarefa
Done: 17
registrando nova tarefa
Done: 20
registrando nova tarefa
Done: 21
registrando nova tarefa
Done: 19
registrando nova tarefa
Done: 23
registrando

In [ ]:
from concurrent.futures import as_completed

In [ ]:
for future in as_completed(futures):
    print(future.result())

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


In [ ]:
""" new_scraper = scraper_registry.get('https://noticias.r7.com/brasilia/comissao-do-senado-sabatina-indicados-de-lula-ao-bc-na-proxima-terca-feira-05122024/') """

" new_scraper = scraper_registry.get('https://noticias.r7.com/brasilia/comissao-do-senado-sabatina-indicados-de-lula-ao-bc-na-proxima-terca-feira-05122024/') "

In [ ]:
""" new_scraper.extract('https://noticias.r7.com/brasilia/comissao-do-senado-sabatina-indicados-de-lula-ao-bc-na-proxima-terca-feira-05122024/') """

" new_scraper.extract('https://noticias.r7.com/brasilia/comissao-do-senado-sabatina-indicados-de-lula-ao-bc-na-proxima-terca-feira-05122024/') "

In [ ]:
""" scraper = R7Scraper()
scraper.extract(article_link='https://noticias.r7.com/brasilia/comissao-do-senado-sabatina-indicados-de-lula-ao-bc-na-proxima-terca-feira-05122024/') """

""" scraper = G1Scraper()
scraper.extract(article_link="https://g1.globo.com/saude/noticia/2024/12/04/o-que-acontece-com-o-corpo-quando-a-vitamina-d-esta-baixa.ghtml") """

""" scraper = BandScraper()
scraper.extract(article_link='https://bandnewstv.uol.com.br/2024/12/05/mauro-cid-vai-prestar-novo-depoimento-a-policia-federal-nesta-quinta-feira/') """

" scraper = BandScraper()\nscraper.extract(article_link='https://bandnewstv.uol.com.br/2024/12/05/mauro-cid-vai-prestar-novo-depoimento-a-policia-federal-nesta-quinta-feira/') "